# Importando

In [ ]:
!pip install pandas openpyxl nltk

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
import string  # Para lidar com pontuação
import re      # Para expressões regulares (remoção de números)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
try:
    df_fake = pd.read_csv("Fake.csv", on_bad_lines='skip', sep=',')
    df_true = pd.read_csv("True.csv", on_bad_lines='skip', sep=',')
except Exception as e:
    print(f"An error occurred while reading the CSV files: {e}")

# Verficações rápidas


In [ ]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
df_true.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [ ]:
df_fake.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

Não tem linhas nulas

In [ ]:
df_fake.isnull().sum()

,0
title,0
text,1
subject,1
date,1


In [ ]:
df_true.isnull().sum()

,0
title,0
text,0
subject,0
date,0


# Removendo Duplicatas

Fake

In [ ]:
df_fake.duplicated().sum()

np.int64(3)

In [ ]:
len(df_fake)

19015

In [ ]:
df_fake = df_fake.drop_duplicates()

In [ ]:
len(df_fake)

19012

In [ ]:
df_fake.duplicated().sum()

np.int64(0)

True

In [ ]:
df_true.duplicated().sum()

np.int64(206)

In [ ]:
len(df_true)

21417

In [ ]:
df_true.drop_duplicates(inplace=True)

In [ ]:
df_true.duplicated().sum()

np.int64(0)

# Adicionando coluna de verdadeiro ou falso


In [ ]:
df_true['is_fake'] = False
df_fake['is_fake'] = True

In [ ]:
df_fake.head()

,title,text,subject,date,is_fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",True
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",True
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",True
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",True
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",True


# Juntando Dataframes

In [ ]:
df = pd.concat([df_true, df_fake], ignore_index=True)

In [ ]:
df.columns

Index(['title', 'text', 'subject', 'date', 'is_fake'], dtype='object')

In [ ]:
len(df)

40223

In [ ]:
df.to_excel('Noticias.xlsx', index=False)

# Limpeza

In [ ]:
# --- Configuração Inicial ---

# 1. Carrega a lista de stopwords em português
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Erro: Pacotes NLTK 'stopwords' ou 'punkt' não encontrados.")
    print("Por favor, rode: nltk.download('stopwords') e nltk.download('punkt')")
    exit()

# 2. Define os nomes de entrada e saída
arquivo_entrada = 'Noticias.xlsx'
arquivo_saida_csv = 'Noticias_limpas.csv'
colunas_para_limpar = ['title', 'text']

# --- Função de Limpeza de Texto

def limpar_texto(texto):
    # 1. Verifica se o dado é uma string (ignora valores nulos/NaN)
    if not isinstance(texto, str):
        return "" # Retorna string vazia para dados nulos ou não-texto

    # 2. Conversão para minúsculas
    texto = texto.lower()

    # 3. Remoção de pontuação
    translator = str.maketrans('', '', string.punctuation)
    texto = texto.translate(translator)

    # (Extra) Remoção de números (muito útil para NLP)
    # texto = re.sub(r'\d+', '', texto)

    # 4. Tokenização
    tokens = word_tokenize(texto, language='english')

    # 5. Remoção de Stopwords
    tokens_limpos = []
    for palavra in tokens:
        if palavra not in stop_words:
            tokens_limpos.append(palavra)

    # 6. Juntar os tokens de volta em uma única string limpa
    return " ".join(tokens_limpos)

# --- Processo Principal ---

print(f"Carregando o dataset '{arquivo_entrada}'...")
try:
    # Lê o arquivo Excel
    df = pd.read_excel(arquivo_entrada)
except FileNotFoundError:
    print(f"Erro: O arquivo '{arquivo_entrada}' não foi encontrado.")
    print("Certifique-se de que o script do seu amigo já foi executado.")
    exit()
except Exception as e:
    print(f"Erro ao ler o arquivo Excel: {e}")
    exit()

print("Dataset carregado. Iniciando limpeza...")

# Loop para aplicar a limpeza em todas as colunas de texto necessárias
for coluna in colunas_para_limpar:
    if coluna in df.columns:
        print(f"Limpando coluna: '{coluna}'...")
        # Cria uma nova coluna (ex: 'title_limpo') com o resultado
        df[f'{coluna}_limpa'] = df[coluna].apply(limpar_texto)
    else:
        print(f"Aviso: Coluna '{coluna}' não encontrada. Pulando...")

print("Limpeza concluída.")

# --- Exibe um exemplo ---
print("\n--- Exemplo de Dados Limpos ('text_limpa') ---")
print(df['text_limpa'].head(2).to_string(index=False))
print("-------------------------------------------------")


# FINAL

# Salva o DataFrame COMPLETO (colunas originais + limpas + 'is_fake')
try:
    df.to_csv(arquivo_saida_csv, index=False, encoding='utf-8')
    print(f"\nSucesso! O conjunto de dados limpo foi salvo em '{arquivo_saida_csv}'.")
    print("Este arquivo contém os textos limpos e a coluna 'is_fake'.")
    print("Pronto para entregar ao Engenheiro de ML (PRIMO)! 👍")
except Exception as e:
    print(f"\nErro ao salvar o arquivo CSV: {e}")

Carregando o dataset 'Noticias.xlsx'...
Dataset carregado. Iniciando limpeza...
Limpando coluna: 'title'...
Limpando coluna: 'text'...
Limpeza concluída.

--- Exemplo de Dados Limpos ('text_limpa') ---
washington reuters head conservative republican...
washington reuters transgender people allowed f...
-------------------------------------------------

Sucesso! O conjunto de dados limpo foi salvo em 'Noticias_limpas.csv'.
Este arquivo contém os textos limpos e a coluna 'is_fake'.
Pronto para entregar ao Engenheiro de ML (PRIMO)! 👍


In [ ]:
df_limpo = pd.read_csv("Noticias_limpas.csv")
df_limpo.head()

,title,text,subject,date,is_fake,title_limpa,text_limpa
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",False,us budget fight looms republicans flip fiscal ...,washington reuters head conservative republica...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",False,us military accept transgender recruits monday...,washington reuters transgender people allowed ...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",False,senior us republican senator let mr mueller job,washington reuters special counsel investigati...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",False,fbi russia probe helped australian diplomat ti...,washington reuters trump campaign adviser geor...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",False,trump wants postal service charge much amazon ...,seattlewashington reuters president donald tru...


# Definindo os Dados Limpos para Treino e Teste

In [ ]:
# Carregando dataframe que FOI limpo na Sprint 1
df_limpo = pd.read_csv("Noticias_limpas.csv")

# Garantindo que não há valores nulos (NaN) nas colunas de texto limpo
df_limpo['title_limpa'] = df_limpo['title_limpa'].fillna('')
df_limpo['text_limpa'] = df_limpo['text_limpa'].fillna('')

# 1. Criar a coluna 'full_text_limpo' a partir das colunas JÁ LIMPAS
df_limpo["full_text_limpo"] = df_limpo["title_limpa"] + " " + df_limpo["text_limpa"]

# 2. Definir X (features) e Y (target) corretamente
X = df_limpo["full_text_limpo"]
Y = df_limpo["is_fake"]

# 3. Refazer a divisão Treino/Teste com os dados limpos
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,       # 20% para teste
    random_state=42,     # Garante reprodutibilidade
    stratify=Y           # Mantém a proporção de True/False nos dois datasets
)

print("Divisão Treino/Teste refeita com DADOS LIMPOS.")
print(f"Tamanho de X_train (textos de treino): {len(X_train)}")
print(f"Tamanho de X_test (textos de teste): {len(X_test)}")

Divisão Treino/Teste refeita com DADOS LIMPOS.
Tamanho de X_train (textos de treino): 32178
Tamanho de X_test (textos de teste): 8045


# Vetorização TF-IDF

In [ ]:
# 1. Inicializar o vetorizador TF-IDF
# Nota: Não precisamos de 'lowercase=True' ou 'stop_words='english''
# porque os dados em X_train JÁ estão limpos pela Sprint 1.
vectorizer = TfidfVectorizer()

# 2. Aprender o vocabulário e transformar os dados de TREINO (fit_transform)
# Isso cria a matriz TF-IDF para o treino
X_train_tfidf = vectorizer.fit_transform(X_train)

# 3. APENAS transformar os dados de TESTE (transform)
# Usamos o mesmo vocabulário aprendido com os dados de treino
X_test_tfidf = vectorizer.transform(X_test)

print("Vetorização concluída.")
print(f"Formato da matriz de Treino (X_train_tfidf): {X_train_tfidf.shape}")
print(f"Formato da matriz de Teste (X_test_tfidf): {X_test_tfidf.shape}")

Vetorização concluída.
Formato da matriz de Treino (X_train_tfidf): (32178, 187785)
Formato da matriz de Teste (X_test_tfidf): (8045, 187785)


# Treinamento e Avaliação

In [ ]:
# 1. Inicializar o modelo de Regressão Logística
# random_state=42 garante que o resultado seja o mesmo se rodarmos de novo
model = LogisticRegression(random_state=42)

print("Iniciando o treinamento do modelo...")

# 2. Treinar o modelo
# O 'fit' usa a matriz TF-IDF de treino (X_train_tfidf)
# e os rótulos de treino (Y_train - True/False)
model.fit(X_train_tfidf, Y_train)

print("Treinamento concluído.")

# 3. Fazer previsões nos dados de TESTE
# Usamos o modelo treinado para prever os rótulos de X_test_tfidf
Y_pred = model.predict(X_test_tfidf)

print("Previsões feitas nos dados de teste.")

# 4. Avaliar os resultados
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print("\\n--- Avaliação do Modelo ---")
print(f"Acurácia: {accuracy * 100:.2f}%")
print("\\nRelatório de Classificação:")
print(report)

Iniciando o treinamento do modelo...
Treinamento concluído.
Previsões feitas nos dados de teste.
\n--- Avaliação do Modelo ---
Acurácia: 98.45%
\nRelatório de Classificação:
              precision    recall  f1-score   support

       False       0.98      0.99      0.99      4242
        True       0.99      0.98      0.98      3803

    accuracy                           0.98      8045
   macro avg       0.98      0.98      0.98      8045
weighted avg       0.98      0.98      0.98      8045



# Treino e Teste

Colunas X , para "estudo" = Text + Title

Coluna Y, para "previsão" = is_fake

In [ ]:
df["full_text"] = df["title"] + " " + df["text"] #criando uma coluna nova que junta o titulo mais o texto
X = df["full_text"]
Y = df["is_fake"]

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    # Stratify é CRUCIAL aqui, pois garante que a proporção 50/50 (ou a que for)
    # seja mantida tanto no treino quanto no teste.
    stratify=Y
)

In [ ]:
print("Divisão Treino/Teste Concluída!")
print(f"Proporção de 'Fake News' (1) no Treino: {Y_train.mean():.2f}")
print(f"Proporção de 'Fake News' (1) no Teste: {Y_test.mean():.2f}")

Divisão Treino/Teste Concluída!
Proporção de 'Fake News' (1) no Treino: 0.47
Proporção de 'Fake News' (1) no Teste: 0.47
